In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import numpy as np
from scipy.io import wavfile
import tensorflow as tf
import utilities as util

import matplotlib.pylab as plt 

import warnings
warnings.filterwarnings("ignore", category=wavfile.WavFileWarning)

In [2]:
%%time
nps_uniq, dfreqs, notes = util.precompute_spect()
smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))
note_freq = util.frequencies('A0', 'D8')
sample_files, target_vals = util.gather_samples(util.note_names('C1', 'G1'), 2) # Guitar, Trombone, Piano
init = tf.global_variables_initializer()


CPU times: user 9.53 ms, sys: 634 µs, total: 10.2 ms
Wall time: 9.88 ms


In [3]:
%%time

all_specs = []
for sf in sample_files:
    s = util.preprocess(sf, nps_uniq, dfreqs, notes, smallest_nps, note_freq)
    all_specs.append(s)

CPU times: user 15min 45s, sys: 1.82 s, total: 15min 47s
Wall time: 15min 49s


In [5]:
X = tf.placeholder(tf.float16, shape=(None,n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [6]:
X.shape

TensorShape([Dimension(None), Dimension(7744)])

In [7]:
%%time

n_hidden1 = 2500
n_hidden2 = 500
n_outputs = 3

with tf.name_scope("network"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu) # Play with activations
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu) # Play with activations
    net_results = tf.layers.dense(hidden2, n_outputs, name="outputs")

CPU times: user 30.4 ms, sys: 2 µs, total: 30.4 ms
Wall time: 30.3 ms


In [8]:
%%time

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=net_results)
    loss = tf.reduce_mean(xentropy, name="loss")

CPU times: user 3.01 ms, sys: 0 ns, total: 3.01 ms
Wall time: 2.85 ms


In [9]:
%%time
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

CPU times: user 109 ms, sys: 4 µs, total: 109 ms
Wall time: 108 ms


In [10]:
%%time
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(tf.cast(net_results, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

CPU times: user 3.03 ms, sys: 3 µs, total: 3.03 ms
Wall time: 2.81 ms


In [11]:
%%time

spectrograms = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    spectrograms = sess.run(all_specs)

CPU times: user 3min 7s, sys: 2.8 s, total: 3min 10s
Wall time: 3min 9s


In [12]:
%%time
new_specs22 = []
for i in range(len(all_specs)):
    new_spectrograms = []
    end = len(np.trim_zeros(spectrograms[i].sum(1), 'b'))
    new_spectrograms = np.concatenate(spectrograms[i][:,:end]).ravel()
#     print(new_spectrograms.shape)
    new_specs22.append(new_spectrograms)

(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
(7744,)
CPU times: user 6.98 ms, sys: 1 ms, total: 7.98 ms
Wall time: 5.96 ms


In [13]:
%%time
saver = tf.train.Saver()

n_epochs = 10
batch_size = 3
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
#         for iteration in range(len(new_specs22) // batch_size):
        X_batch = new_specs22
        Y_batch = target_vals
        sess.run(training_op, feed_dict={X: np.stack(X_batch),
                                         y: np.array(Y_batch)})
#         acc_train = accuracy.eval(feed_dict={X: X_batch, y: [0,1,2]})
        acc_val = accuracy.eval(feed_dict={X: X_batch, y: Y_batch})
        print(epoch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./FirstNetwork.ckpt")

0 Train accuracy: 0 Val accuracy: 1.0
1 Train accuracy: 0 Val accuracy: 1.0
2 Train accuracy: 0 Val accuracy: 1.0
3 Train accuracy: 0 Val accuracy: 1.0
4 Train accuracy: 0 Val accuracy: 1.0
5 Train accuracy: 0 Val accuracy: 1.0
6 Train accuracy: 0 Val accuracy: 1.0
7 Train accuracy: 0 Val accuracy: 1.0
8 Train accuracy: 0 Val accuracy: 1.0
9 Train accuracy: 0 Val accuracy: 1.0
CPU times: user 19.8 s, sys: 480 ms, total: 20.3 s
Wall time: 21.7 s


In [14]:
new_sample_files = []
new_target_vals = []
new_location = '../samples/'
new_instruments = ['Trombone', 'Guitar', 'Piano']
for i, new_name in enumerate(new_instruments):
    for new_note in (util.note_names('C1', 'F1')):
        for j in range(2, 3):
            new_file_name = new_location + new_name+"/"+new_name +"_"+new_note[:-1]+"("+str(j)+").wav"
            new_sample_files.append(new_file_name)
            new_target_vals.append(i)

In [15]:
%%time
# Preprocess all of the sample files
# Create lists of all of the frequencies, times and spectrograms
new_all_freqs = note_freq[:-1]
new_all_specs = []
new_all_names = []
for sf in new_sample_files:
    # TODO: TF_SPEC needs all of these constants. Is there a way that we can reduce the parameters?
    new_s = util.preprocess(sf, nps_uniq, dfreqs, notes, smallest_nps, note_freq)
    new_all_specs.append(s)

CPU times: user 5min 56s, sys: 429 ms, total: 5min 57s
Wall time: 5min 58s


In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    spectrograms2 = sess.run(new_all_specs)

In [17]:
%%time
new_specs33 = []
for i in range(len(new_all_specs)):
    new_spectrograms2 = []
    end = len(np.trim_zeros(spectrograms2[i].sum(1), 'b'))
    new_spectrograms2 = np.concatenate(spectrograms2[i][:,:end]).ravel()
    new_specs33.append(new_spectrograms2)

CPU times: user 1.1 ms, sys: 1 µs, total: 1.1 ms
Wall time: 812 µs


In [18]:
depth = 3
with tf.Session() as sess:
    saver.restore(sess, "./FirstNetwork.ckpt")
    Z = net_results.eval(feed_dict={X: new_specs33})

INFO:tensorflow:Restoring parameters from ./FirstNetwork.ckpt


In [20]:
Z

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float16)

In [21]:
new_target_vals

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]